# CH-07 年報問答機器人

## 7-2 取得年報

### 1️⃣  匯入套件

In [ ]:
import requests
from bs4 import BeautifulSoup

### 2️⃣ 建立函式-取得年報

In [ ]:
def annual_report(id,y):
  url = 'https://doc.twse.com.tw/server-java/t57sb01'
  # 建立 POST 請求的表單
  data = {
      "id":"",
      "key":"",
      "step":"1",
      "co_id":id,
      "year":y,
      "seamon":"",
      "mtype":'F',
      "dtype":'F04'
  }
  try:
    # 發送 POST 請求
    response = requests.post(url, data=data)
    # 取得回應後擷取檔案名稱
    link=BeautifulSoup(response.text, 'html.parser')
    link1=link.find('a').text
    print(link1)
  except Exception as e:
    print(f"發生{e}錯誤")
  # 建立第二個 POST 請求的表單
  data2 = {
      'step':'9',
      'kind':'F',
      'co_id':id,
      'filename':link1 # 檔案名稱
  }
  try:
    # 發送 POST 請求
    response = requests.post(url, data=data2)
    link=BeautifulSoup(response.text, 'html.parser')
    link1=link.find('a')
    # 取得 PDF 連結
    link2 = link1.get('href')
    print(link2)
  except Exception as e:
    print(f"發生{e}錯誤")
  # 發送 GET 請求
  try:
    response = requests.get('https://doc.twse.com.tw' + link2)
    # 取得 PDF 資料
    with open(y + '_' + id + '.pdf', 'wb') as file:
        file.write(response.content)
    print('OK')
  except Exception as e:
    print(f"發生{e}錯誤")

### 3️⃣ 呼叫函式

In [ ]:
annual_report('2330','113')

2023_2330_20240604F04.pdf
/pdf/2023_2330_20240604F04_20250211_142706.pdf
OK


## 7-3 年報問答

langchain 以更新至新版寫法，舊版可參考：
https://colab.research.google.com/drive/16x0mUitJjH0PZx7kk2Kew2MSlkB9G_lA

###4️⃣ 安裝相關套件

In [ ]:
!pip install pdfplumber langchain langchain_openai langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.33
    Uninstalling langchain-core-0.3.33:
      

###  5️⃣ 匯入相關套件

In [ ]:
import os
import getpass
from langchain_community.document_loaders import PDFPlumberLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

### 6️⃣ 設定環境變數和建立 OpenAI 模型

In [ ]:
os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')
llm_model = ChatOpenAI(model="gpt-4o")

OpenAI API Key:··········


### 7️⃣ 建立函式-建立向量資料庫

In [ ]:
def build_vector_db(file_path, size, overlap):
    loader = PDFPlumberLoader(file_path)
    doc = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=size,
        chunk_overlap=overlap)
    docs = text_splitter.split_documents(doc)
    db = InMemoryVectorStore.from_documents(docs, OpenAIEmbeddings())
    return db

### 8️⃣  呼叫函式

In [ ]:
db = build_vector_db('/content/113_2330.pdf', 500, 50)

### 9️⃣ 查詢相關資料

In [ ]:
similarity_docs = db.similarity_search("研發的產品", k=5)
for i in similarity_docs:
    print(i.page_content)
    print('_________')

和先進封裝解決方案方面強大的技術領先，我們能夠掌握更多的產業成長機會。
我們專注於公司的業務基本面，有目標地執行我們的全球製造足跡策略，以支持客戶成長與增加他們的信任。我
劉德音 魏哲家
們將繼續推動台積公司在全球所有晶圓廠的數位卓越化，並致力實現全方位的智慧化和自動化製造。無論我們在
董事長 總裁
何處營運，我們都決心要成為具有最高效率和最具成本效益的半導體製造者。
010 011
_________
米製程技術在民國一百一十二年下半年邁入高度量產且成長強勁。我們亦持續強化包括N3E、N3P和N3X在內的 儘管海外晶圓廠的起始成本高於台積公司在台灣的晶圓廠，但我們有信心控制及最小化成本差距，因而能夠持續
N3製程技術，並預計在眾多客戶的強勁需求支持下，N3製程技術在民國一百一十三年與未來幾年中將貢獻更多 將股東價值最大化。
的營收。
006 007
_________
5.1.2 客戶產品用途 ●台 積公司的導線與封裝技術整合方案3DFabric®在支 5.2.2 民國一百一十二年研究發展成果
援CoWoS®-S和CoWoS®-L上最新一代高頻寬記憶體
民國一百一十二年，台積公司為528個客戶生產1萬1,895種不同的產品，應用範圍涵括整個電子應用產業，包括人工智 研究發展組織卓越成果
HBM3E方面取得了進一步進展；在公司繼續保持InFO_
慧和高效能運算伺服器、有線與無線通訊系統產品、汽車與工業用設備、個人電腦與其周邊產品、資訊應用產品，以及 ●2 奈米製程技術
PoP Gen-8封裝大批量製造業領先地位的同時，InFO_
包括數位電視、遊戲機、數位相機等消費性電子、智慧物聯網及穿戴式裝置，與其他許多產品與應用。 民國一百一十二年，台積公司2奈米研發著重於基礎製
PoP Gen-9也完成行動應用認證。此外，InFO_oS Gen-5
程制定、良率精進、電晶體及導線效能改善以及可靠性
成功通過認證，提供更大的ASIC面積、更多的晶片分區
終端電子產品的快速演進，將促使客戶採用台積公司創新的技術與服務以追求差異化，同時也會更促進台積公司自身 評估。本年度內，重要客戶完成矽智財設計，並開始做驗
整合、更大的封裝尺寸和更高的頻寬。
的技術發展。台積公司始終深信成功將屬於產業趨勢的領導者而非追隨者。 證。台積公司並且發展低阻值重置導線層、超高效能金
●在特殊製程技

### 🔟  匯入問答相關套件

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA

### 1️⃣1️⃣  建立函式-問答程式

In [ ]:
# 提示模板
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "你是一個根據年報資料與上下文作回答的助手,"
     "如果有明確數據或技術(產品)名稱可以用數據或名稱回答,"
     "回答以繁體中文為主。"
     "{context}"),
    ("human","{question}")])

# 建立問答函式
def question_and_answer(question):
    retrievalQA = RetrievalQA.from_llm(
        llm=llm_model,
        prompt=prompt,
        return_source_documents=True,
        retriever=db.as_retriever(
        search_kwargs={'k':8}))
    answer = retrievalQA.invoke(question)
    return answer

### 1️⃣2️⃣ 建立迴圈進行問答

In [ ]:
while True:
    question = input("輸入問題:")
    if not question.strip():
        break
    result = question_and_answer(question)
    print(result['result'])
    print('_________')
    #print(result["source_documents"])

輸入問題:公司是否有明確成長?
根據台積公司的年報資料，民國112年的全年合併營收為新台幣2兆1,617億4,000萬元，較前一年的2兆2,638億9,000萬元減少4.5%；稅後淨利為新台幣8,385億元，較前一年的1兆165億3,000萬元減少17.5%。這些數據顯示公司在最近一年經歷了營收和淨利的下降，因此從數據上看，近期的成長並不明顯。

此外，毛利率與營業利益率也有所下降，分別從59.6%降至54.4%及從49.5%降至42.6%。這些因素顯示出在成長方面可能面臨挑戰，因此整體來看，台積公司並沒有明確的成長趨勢。
_________
輸入問題:公司未來一年的計畫
根據年報資料，台積公司未來一年的計畫主要集中在以下幾個方面：

1. **資本支出**：台積公司預計在民國112年及113年的資本支出將有顯著的增加，特別是在生產、研發及廠務設備方面。預計單位為新台幣2,032,488,955仟元，其中針對十二吋晶圓的產能增加約80萬片。

2. **現金流動性**：雖然未來一年現金流動性分析不適用，但公司表示無現金流動性不足之情形，並計畫持續審慎評估其資金運用計劃。

3. **永續發展與綠色債券**：台積公司已發行綠色債券以支應公司綠建築及環保相關之資本支出。未來將持續推進與環保相關的投資計畫，例如112年度第2期無擔保普通公司債計畫預定於民國113年第一季開始執行。

4. **風險管理**：公司將進一步強化風險管理架構，並依據《ISO 31000: 2018風險管理系統》及《COSO委員會之企業風險管理-整合架構》來推動這些工作。

這些計畫顯示了台積公司對於未來展望的信心與積極的業務發展策略。
_________
輸入問題:


## 7-4 年報總結與分析

### 1️⃣3️⃣ 回答結果及原始資料

In [ ]:
from langchain.chains.summarize import load_summarize_chain

### 1️⃣4️⃣ 總結原始資料

In [ ]:
# 建立關鍵字串列
key_word = ['正在開發的產品及銷售狀況',
            '市場策略的調整或變化',
            '公司預期未來展望',
            '總營收、稅前淨利的成長或變動分析',
            '國際競爭以及海外市場銷售情形']

data_list = []
for word in key_word:
    data = db.similarity_search(word, k=3)
    # 整合 Document 串列
    data_list += data

# 建立提示訊息串列
prompt_template = [("system","你的任務是對年報資訊進行摘要總結。"
                    "以下為提供的年報資訊：{text},"
                    "請給我重點數據, 如銷售增長情形、營收變化、開發項目等,"
                    "最後請使用繁體中文輸出報告")]
prompt = ChatPromptTemplate.from_messages(messages=prompt_template)

### 1️⃣5️⃣  呼叫函式

In [ ]:
refine_chain = load_summarize_chain(llm=llm_model,
                                    chain_type='stuff',
                                    prompt=prompt)
summary_refine = refine_chain.invoke({"input_documents": data_list})
print(summary_refine['output_text'])

### 年報資訊摘要

#### 營收與財務績效
- **營業收入**：2022年度營業收入淨額為新台幣2,161.7億元，較2021年度的2,263.9億元減少約4.5%。
- **營業毛利**：營業毛利下降13%，從2021年的1,300.4億元降至2022年的1,130.6億元。
- **淨利**：2022年度淨利為838.5億元，較前年度下降約18%。

#### 環境、社會及公司治理（ESG）
- **員工安全與健康**：公司定期實施安全與健康教育，並提供安全的工作環境，詳細措施見於報告中「安全與衛生」部分。
- **環境管理**：本公司建立了符合產業特性的環境管理制度，並遵循相關法律規範，有關管理制度及驗證情形可查閱「環保、安全與衛生管理」部分。
- **能源使用**：公司致力於提升能源利用效率以及使用再生材料，具體政策及措施詳見「氣候變遷與能源管理」部分。
- **氣候變遷風險評估**：根據TCFD框架，進行氣候變遷風險與機會的識別及評估，並針對短期、中期及長期的潛在影響採取應對措施。

#### 技術與研發
- **新產品開發**：公司在高效能運算、AI、物聯網及車用電子等領域持續進行技術研發，並推出新一代環保材料及電晶體技術，期望在下一年度進入生產。
- **專利管理**：公司積極進行專利部件的擴建及保護，以保持在半導體產業的技術領先地位。

#### 其他財務資訊
- **合約負債**：截至2022年12月31日合約負債為新台幣527.4億元，較2021年下降，主要因履約義務滿足時間的差異。
- **長期投資**：長期投資有所增加，主要為非流動金融資產的增長。

#### 綜合分析
- 公司面臨巨大的市場波動及成本壓力，包括工人短缺、供應鏈中斷及天然災害等風險，這可能影響營運效率及未來的產擴計畫。
- 繼續推動永續發展與產品創新，未來將專注於技術進步及應對氣候變遷帶來的挑戰，旨在增強企業竞争力及客戶信任。

這份年報摘要提供了公司在營運、環境管理及未來發展策略的概覽，顯示出積極應對市場挑戰及推動永續發展的努力。


### 1️⃣6️⃣  提取關鍵字
使用 MMR 搜尋方法

In [ ]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser, StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence

word_prompt = PromptTemplate.from_template(
     "從{input}聯想出4個與年報分析有關的重要關鍵字,"\
     "請確保回答具有具有關聯性、多樣性和變化性。 \n "
     "僅回覆關鍵字, 並以半形逗號與空格來分隔。不要加入其他內容")

word_chain = word_prompt | llm_model | CommaSeparatedListOutputParser()
print(word_chain.invoke({"input": "公司的營運狀況如何?"}))

['營收增長', '毛利率', '現金流', '債務比率']


### 1️⃣7️⃣ 設定 AI 角色讓其分析報告

In [ ]:
data_prompt = PromptTemplate.from_template(
    "你現在是一位專業的股票分析師,"
    "會以詳細、嚴謹的角度針對 {key_words} 進行年報分析,"
    "請提及關於營收、是否成長以及利潤等重要數字,"
    "最後生成一份專業的趨勢分析報告。"
    "以下為年報資料：{data_content}")

data_chain = data_prompt | llm_model | StrOutputParser()

### 1️⃣8️⃣ 整合函式

In [ ]:
def analyze_chain(input):
    # 搜尋「問題」的相關資料
    data = db.max_marginal_relevance_search(input, fetch_k=5, k=2)

    # 第一個 Chain 元件, 建立「關鍵字」串列
    word_list = word_chain.invoke({"input": input})

    # 搜尋「關鍵字」的相關資料
    for word in word_list:
      data += db.max_marginal_relevance_search(word, fetch_k=5, k=2)
    word_list.append(input)

    # 第二個 Chain 元件, 生成分析報告
    result = data_chain.invoke({'key_words':word_list,'data_content':data})

    return result

### 1️⃣9️⃣ 呼叫函式

In [ ]:
input = '公司的營收狀況如何？'
analyze_report = analyze_chain(input)
print(analyze_report)

## 年報分析報告

### 基本資訊
本報告針對台灣積體電路製造股份有限公司（以下簡稱「台積電」）的112年度年報資料進行分析，主要聚焦於以下幾個方面：
1. 營收增長
2. 毛利率
3. 費用控制
4. 資本運用
5. 營收狀況

### 一、營收增長
台積電112年度的淨利為新台幣837,767,517仟元，相較於前年的1,016,900,515仟元，顯示年度淨利減少了約18%。綜合損益總額從前年的1,059,124,890仟元下降至830,509,542仟元，減少幅度達22%。這表明公司的收入在報告期內出現了衰退。營業收益淨額增加，主要源自於處分及報廢不動產、廠房及設備的淨益所致；然而，淨利下降顯示出整體營收面對挑戰。

### 二、毛利率
台積電並未明確提供毛利率數據，但毛利率可根據營收與費用結構推測。考慮到營業外收入的增加（例如利息收入及子公司、關聯企業的損益份額），這部分的增長可能有助於穩定公司的總毛利水平。此外，但由於淨利顯著下降，且前已遞減的綜合損益總額，這暗示著毛利率需持續觀察。

### 三、費用控制
根據年報，研究發展費用為178,725,098仟元，管理費用39,890,037仟元，行銷費用5,118,396仟元。儘管有大規模的投資於研發（顯示出公司對未來技術的堅持），相關的營業收益卻未能有效轉化為利潤，顯示出費用控制的壓力。因此，儘管台積電在特定技術領域持續投入，整體的盈利能力仍然受到挑戰。

### 四、資本運用
在資本運用方面，台積電的資本支出尚未顯著影響其現金流的可持續性。根據文獻中的資本運用計劃及執行進度顯示，資金運用仍依計劃順利進行，某些項目出現延後現象（如政府發放的預算撥款），但對股東權益未造成重大影響。

### 五、公司的營收狀況
從綜合損益總額下滑的趨勢來看，台積電在當年度的營收狀況顯示出一定的風險。毛利率的可能下降、淨利縮減以及費用控制的挑戰，令投資者須謹慎看待未來的盈利預期。

### 總結與趨勢分析
整體來看，台積電在112年度營運表現出現衰退。雖然收入增長來自特定業務（如處分資產的收益），但整體獲利和毛利均顯示下滑跡象。未來，公司需要強化其費用控制與營運效率，並探討提高毛利率的可行策略。此外，調整資本運用以因應不穩定的營收表現將是接下來的重要課題。

在全球經濟環境與技術需求持續快速變化的背景下，台積電需逐步調整